### **Improving custom model**

In [1]:
# DL needs
import tensorflow as tf
import keras as kr

# Data needs
import pandas as pd
from sklearn.model_selection import train_test_split

# Numerical computation needs
import numpy as np

# plotting needs
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# ensuring reproducibility
random_seed=42
tf.random.set_seed(random_seed)

2025-03-11 21:18:58.655528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# creating a CNN model (baseline: tinyVGG)
@kr.saving.register_keras_serializable(package="tinyVGG")

class tinyVGG(kr.Model):
    def __init__(self,lr=0.001,metrics=['accuracy'],**kwargs):
        # calling constructor of kr.Model() class
        super().__init__(**kwargs)

        # defining model layers

        ### CONV-POOL 1
        self.cnn1 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,padding='valid',
                                           input_shape=(224,224,3),name='conv_1')
        self.cnn2 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,name='conv_2')
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=2,
                                               padding='valid',name='pool_1')
        
        ### CONV-POOL 2
        self.cnn3 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,name='conv_3')
        self.cnn4 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,name='conv_4')
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=2,name='pool_2')

        ### FLATTEN
        self.flatten1 = tf.keras.layers.Flatten(name='flatten_1')

        ### output layer
        self.output_layer = tf.keras.layers.Dense(10,name='output_layer')


        # layers:
        self._layers = [self.cnn1, self.cnn2, self.pool1,
                       self.cnn3, self.cnn4, self.pool2,
                       self.flatten1,
                       self.output_layer
                       ]
        
        # activations
        self.activations = {
            'conv_1':'relu',
            'conv_2':'relu',
            'pool_1':None,
            'conv_3':'relu',
            'conv_4':'relu',
            'pool_2':None,
            'flatten_1':None,
            'output_layer':'softmax'
        }

        # outputs & tracking
        self.track_layer_output = False
        self.track_activation_output = False
        self.layer_outputs=[]
        self.activation_outputs=[]
        

        ### other hyperparams
        self.loss_fn = 'categorical_crossentropy',
        self.model_optimizer = kr.optimizers.Adam(learning_rate=lr)
        self.loss_metrics = metrics
        
        ### build the model
        self.build()

    def build(self):
        inputs=tf.keras.Input(shape=[224,224,3])
        x=self._layers[0](inputs)
        for layer in self._layers[1:]:
            x=layer(x)


    def compile(self):
        super().compile(optimizer=self.model_optimizer,loss=self.loss_fn,metrics=self.loss_metrics)

    
    def call(self,inputs):
        x = inputs
        layer_outputs=[]
        activation_outputs=[]

        for layer in self._layers:
            # forward pass and appending output of each layer and activation
            x = layer(x)
            
            if self.track_layer_output:
                layer_outputs.append(x)

            x = self.get_activation(layer_name=layer.name)(x)
            
            if self.track_activation_output:
                activation_outputs.append(x)

        self.layer_outputs=layer_outputs
        self.activation_outputs=activation_outputs

        output = x
        return output


    def get_activation(self,layer_name):
        activation_name = self.activations.get(layer_name,None)
        def activation(input):
            if activation_name == None:
                    return input
            else:
                function = getattr(tf.nn,activation_name,None)
                return input if function is None else function(input)
        return activation
    
model1=tinyVGG(name='model_1')

model1.save('tr.keras')

/home/rudraksha14/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
m2=kr.models.clone_model(model1)
m2.summary()

/home/rudraksha14/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 222, 222, 10)   │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 220, 220, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 110, 110, 10)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 108, 108, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4 (Conv2D)                 │ (None, 106, 106, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 53, 53, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 28090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │       280,910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 283,920 (1.08 MB)

 Trainable params: 283,920 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
m=tf.keras.models.load_model('tr.keras')
m.summary()

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 222, 222, 10)   │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 220, 220, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 110, 110, 10)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 108, 108, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4 (Conv2D)                 │ (None, 106, 106, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 53, 53, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 28090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │       280,910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 283,920 (1.08 MB)

 Trainable params: 283,920 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model1.compile()

TypeError: tinyVGG.compile() got an unexpected keyword argument 'optimizer'

In [ ]:
mo